In [2]:
from apted import APTED
from apted.helpers import Tree
import pandas as pd
import numpy as np
import glob, os
from tqdm import tqdm
import itertools
from bs4 import BeautifulSoup
import bs4
#from collections import defaultdict

from collections import OrderedDict, Callable

class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))



def defaultVal():
    return [[],0]

def makeTree(file):
    with open(file,"r") as f:
        S = f.read()
    S = S.strip()
    S = S.replace("\n","")
    #S = S.replace(" ","")
    S = S.replace("\t","")
    S = S.replace("\r","")
    soup = BeautifulSoup(S, "html.parser")
    return soup

def recursiveChildBfs(bs):
    root = bs
    stack = [root]
    count=0
    parrent = [None]
    while len(stack) != 0:
        node = stack.pop(0)
        pnode = parrent.pop(0)
        if node is not bs:
            if node.name!=None:
                yield node.name+"~"+str(count),pnode
            else:
                yield node.name,pnode
        if hasattr(node, 'children'):
            for child in node.children:
                stack.append(child)
                parrent.append(node.name+"~"+str(count))
        count+=1

def visit(tagdict,c,tree):
    tree+="{"
    tree+=c.split("~")[0]
    for i in tagdict[c][0]:
        tree = visit(tagdict,i,tree)
        tree+="}"
    return tree        

def generateTree(file):
    html = makeTree(file)
    tagdict = DefaultOrderedDict(defaultVal)
    for c,p in recursiveChildBfs(html):
        if c!=None:
            tagdict[p][0].append(c)
            tagdict[p][1]+=1


    tree = "{"
    for x,y in zip(list(tagdict.keys())[1::],list(tagdict.values())[1::]):
        tree+=x.split("~")[0]
        for c in y[0]:
            #tree+="{"
            #tree+=c
            tree = visit(tagdict,c,tree)
            tree+="}"
        tree+="}"
        break
    nNodes = 0
    for x in tagdict.keys():
        nNodes+=tagdict[x][1]
    return tree,nNodes


    
from collections import defaultdict
import pickle
import json

def load_stateMap(name):
    with open(name) as json_file:
        data = defaultdict(factory)
        data2 = json.load(json_file)
        data.update(data2)
    return data

def factory():
    return {"src":"","edges":[],"url":"","start":0}


import networkx as nx

def makeGraph_Qexplore(statedict):
    DG = nx.DiGraph()
    SFG = load_stateMap(statedict)
    count=1
    stateNamesBig = {}
    stateNamesSmall = {}
    edgemap = {}
    indexstate = ""
    indexlist = [x for x in SFG.keys() if SFG[x]["start"]==1]
    statesfile = [file.split("/")[-1].split(".html")[0] for file in glob.glob("./Q_Result/*.html")]
    for x in SFG.keys():
        if x in indexlist:
            stateNamesSmall["index"+str(count)] = x
            stateNamesBig[x] = "index"+str(count)
        else:
            stateNamesSmall["S"+str(count)] = x
            stateNamesBig[x] = "S"+str(count)
        count+=1
    sss = [x for x in set(statesfile).difference(set(SFG.keys())) if x not in ["index","temp"]]
    for x in sss:
        stateNamesSmall["S"+str(count)] = x
        stateNamesBig[x] = "S"+str(count)
        count+=1

    DG.add_nodes_from(stateNamesSmall.keys())
    count = 0
    for state in stateNamesSmall.keys():
        for edge in SFG[stateNamesSmall[state]]['edges']:
            if edge["state"] in statesfile:
                edgemap[edge["action"]] = count
                count+=1
        
    for state in stateNamesSmall.keys():
        edges = [(state,stateNamesBig[edge["state"]]) for edge in SFG[stateNamesSmall[state]]['edges'] if edge["state"] in statesfile]
        #DG.add_edges_from()
        #print(edges)
        DG.add_edges_from(edges)
    return DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap

def makeGraph_crawljax(resultfile,baseurl):
    with open(resultfile) as json_file: 
        data = json.load(json_file)
    DG = nx.DiGraph()
    keys = [k for k in data["states"].keys() if baseurl in data["states"][k]["url"]]
    
    aa = os.listdir("./doms/")
    keys = [k for k in keys if k+".html" in aa]
    DG.add_nodes_from(keys)
    edges = [(edge["from"],edge["to"]) for edge in data["edges"] if edge["from"] in keys and edge["to"] in keys]
    DG.add_edges_from(edges)
    return DG,data

def generateSimplePaths_QExplore(statedict):
    DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap = makeGraph_Qexplore(statedict)
    pathdict = {}
    for node in tqdm(DG.nodes):
        for index in indexlist:
            paths = nx.all_simple_paths(DG, source=stateNamesBig[index], target=node)
            pathlist = []
            for path in paths:
                i=0
                #temp = []
                s = ""
                while(i<len(path)-1):
                    edar = []
                    for edge in SFG[stateNamesSmall[path[i]]]["edges"]:
                        if edge["state"]==stateNamesSmall[path[i+1]]:
                            #temp.append(path[i]+edge["action"])
                            #print("hello")
                            edar.append(path[i])
                    s+="|".join(set(edar))
                    s+="->"
                    i+=1
                temp = splitstring(s)
                pathlist.extend(temp)
        pathdict[node] = pathlist
    return pathdict
        
def generateSimplePaths_crawljax(resultfile,baseurl):
    DG,data = makeGraph_crawljax(resultfile,baseurl)
    pathdict = {}
    for node in tqdm(DG.nodes):
        paths = nx.all_simple_paths(DG, source="index", target=node)
        pathlist = []
        for path in paths:
            i=0
            temp = []
            while(i<len(path)-1):    
                for edge in data["edges"]:
                    if edge["from"]==path[i] and edge["to"]==path[i+1]:
                        temp.append(edge["id"])
                i+=1
            pathlist.append(temp)
        if len(pathlist)>100:
            if node in pathdict:
                pathdict[node].extend(pathlist[0:100])
            else:
                pathdict[node] = pathlist[0:100]
        else:
            if node in pathdict:
                pathdict[node].extend(pathlist)
            else:
                pathdict[node] = pathlist
        #pathdict[node] = pathlist
    return pathdict

from itertools import product
def splitstring(S):
    s = S.split("->")
    i=1
    result = s[0].split("|")
    while(i<len(s)-1):
        result= [x[0]+"->"+x[1] for x in product(result,s[i].split("|"))]
        i+=1
    #temp = [x.split("->") for x in result]
    return [x.split("->") for x in result]

def generateLocalSimplePaths_QExplore(statedict):
    DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap = makeGraph_Qexplore(statedict)
    pathdict = {}
    for node in tqdm(DG.nodes):
        for index in indexlist:
            if node!=stateNamesBig[index]:
                paths = nx.all_simple_paths(DG, source=stateNamesBig[index], target=node)
                pathlist = []
                for path in paths:
                    i=0
                    #temp = []
                    s = ""
                    while(i<len(path)-1):
                        edar = []
                        for edge in SFG[stateNamesSmall[path[i]]]["edges"]:
                            if edge["state"]==stateNamesSmall[path[i+1]]:
                                #temp.append(path[i]+edge["action"])
                                #print("hello")
                                edar.append(path[i]+str(edgemap[edge["action"]]))
                        s+="|".join(set(edar))
                        s+="->"
                        i+=1
                    temp = splitstring(s)
                    pathlist.extend(temp)
                if len(pathlist)>100:
                    if node in pathdict:
                        pathdict[node].extend(pathlist[0:100])
                    else:
                        pathdict[node] = pathlist[0:100]
                else:
                    if node in pathdict:
                        pathdict[node].extend(pathlist)
                    else:
                        pathdict[node] = pathlist
                #try:
                #    print(stateNamesBig[index],node,pathdict["index2"])
                #except:
                #    pass
                
    return pathdict


def getpathSimilarity(path1,path2):
    A = set(path1)
    B = set(path2)
    shared = len(A.intersection(B))
    pathsim = (2*shared)/(len(A)+len(B))
    return pathsim

def calculate_Average_NavigationalDiversity(data):
    keys = [key for key in data.keys() if data[key]!=[]]
    diversity = []
    for key in tqdm(keys):
        paths = data[key]
        sim = []
        for path1,path2 in product(paths,paths):
            sim.append(getpathSimilarity(path1,path2))
        sim = np.array(sim)
        if np.sum(sim==1)==sim.shape[0]:
            diversity.append(1)
        else:
            diversity.append(1-np.min(sim))
    return np.mean(diversity)

def calculate_TestSuitSize(data):
    keys = [key for key in data.keys() if data[key]!=[]]
    Total = 0
    for key in tqdm(keys):
        paths = data[key]
        sim = []
        for path1,path2 in product(paths,paths):
            sim.append(getpathSimilarity(path1,path2))
        sim = np.array(sim)
        Total+=np.sum(sim==0)+1
    return Total

def calculateTestModelSize(data):
    edge = []
    for k in data.keys():
        for path in data[k]:
            edge.extend(path)
    return edge

def generateSimpleStatePaths_QExplore(statedict):
    DG,stateNamesBig,stateNamesSmall,indexlist,SFG,edgemap = makeGraph_Qexplore(statedict)
    pathdict = {}
    for node in tqdm(DG.nodes):
        pathlist = []
        for index in indexlist:
            paths = nx.all_simple_paths(DG, source=stateNamesBig[index], target=node)
            pathlist.extend([path for path in paths])
        if len(pathlist)>100:
            pathlist = pathlist[0:100]
        pathdict[node] = pathlist
    return pathdict
        
def generateSimpleStatePaths_crawljax(resultfile,baseurl):
    DG,data = makeGraph_crawljax(resultfile,baseurl)
    pathdict = {}
    for node in tqdm(DG.nodes):
        paths = nx.all_simple_paths(DG, source="index", target=node)
        AA = [path for path in paths]
        if len(AA)>100:
            pathdict[node] = AA[0:100]
        else:
            pathdict[node] = AA
    return pathdict

def calculateDomDiveristy(x,y,path):
    tree1,n1 = generateTree(x)
    tree2,n2 = generateTree(y)
    t1 = Tree.from_text(tree1)
    t2 = Tree.from_text(tree2)
    apted = APTED(t1, t2)
    ted = apted.compute_edit_distance()
    DD = ted/max(n1,n2)
    with open(path,"w") as file:
        file.write(str(DD))
        
def makeUniqueDom(backup,path):
    from shutil import copyfile,rmtree
    import time
    if os.path.exists(backup):
        #shutil.rmtree(backup)
        #os.mkdir(backup)
        pass
    else:
        os.mkdir(backup)

    doms = os.listdir(path)
    for dom in doms:
        copyfile(os.path.join(path,dom), os.path.join(backup,dom))
    
    for dom1 in tqdm(doms):
        domlist = []
        if dom1 in os.listdir(path) and dom1.split(".")[-1]=="html":
            tree1,n1 = generateTree(os.path.join(path,dom1))
            t1 = Tree.from_text(tree1)
            for dom2 in os.listdir(path):
                if dom2.split(".")[-1]=="html":
                    if dom1!=dom2:
                        tree2,n2 = generateTree(os.path.join(path,dom2))
                        t2 = Tree.from_text(tree2)
                        apted = APTED(t1, t2)
                        ted = apted.compute_edit_distance()
                        if ted<1:
                            domlist.append(dom2)
            for d in domlist:
                os.remove(os.path.join(path,d))
        time.sleep(0.5)

# make Unique STATES

In [3]:
makeUniqueDom("./dombackup","./doms")

100%|██████████| 63/63 [19:37<00:00, 18.69s/it] 


In [4]:
makeUniqueDom("./Qbackup","./Q_Result")

100%|██████████| 39/39 [01:43<00:00,  2.65s/it]


# DOM DIVERSITY
Run process.py

# Navigational Functionality

In [5]:
Qpaths = generateLocalSimplePaths_QExplore("./Q.map")

100%|██████████| 16/16 [00:00<00:00, 1865.90it/s]


In [6]:
import json
with open("qpaths.json", "w") as outfile:  
    json.dump(Qpaths, outfile)

In [7]:
#import json
#with open("paths.json") as json_file:
#    Qpaths = json.load(json_file)

In [8]:
cpaths = generateSimplePaths_crawljax("./result.json","http://192.168.1.68/timeclock")

100%|██████████| 38/38 [00:00<00:00, 893.52it/s]


In [9]:
import json
with open("cpaths.json", "w") as outfile:  
    json.dump(cpaths, outfile)

In [10]:
#Qdiversity = calculate_Average_NavigationalDiversity(Qpaths)
#Qdiversity

# run processpathfile

# Test model size

In [11]:
#Qpaths = generateLocalSimplePaths_QExplore("./Q.map")
len(set(calculateTestModelSize(Qpaths)))

42

In [12]:
len(set(calculateTestModelSize(cpaths)))

29

# Test Suit Size

In [13]:
A = generateSimpleStatePaths_QExplore("./Q.map")

100%|██████████| 16/16 [00:00<00:00, 853.60it/s]


In [14]:
B = generateSimpleStatePaths_crawljax("./result.json","http://192.168.1.68/timeclock")

100%|██████████| 38/38 [00:00<00:00, 4090.22it/s]


In [15]:
calculate_TestSuitSize(A)

100%|██████████| 15/15 [00:00<00:00, 4185.93it/s]


15

In [16]:
calculate_TestSuitSize(B)

100%|██████████| 26/26 [00:00<00:00, 4280.07it/s]


26

# Test Model Size Unique
run process.py

# HTML Coverage

In [17]:
import os
import cryptohash as chash
def traverseTree(tree,unique=set()):
    unique.add(tree.name)
    for x in tree.children:
        traverseTree(x,unique)
    return unique

In [18]:
path = "./Q_Result/"
exclude = [path+"index.html",path+"temp.html"]
states = [file for file in glob.glob(path+"*.html") if file not in exclude]
totaldomq = set()
for state in tqdm(states):
    tree,nodes = generateTree(state)
    tree = Tree.from_text(tree)
    taglist = traverseTree(tree)
    soup = makeTree(state)
    for tags in taglist:
        for tag in soup.find_all(tags):
            totaldomq.add(tag)
len(totaldomq)

100%|██████████| 16/16 [00:00<00:00, 35.14it/s]


962

In [19]:
path = "./doms/"
exclude = [path+"temp.html"]
states = [file for file in glob.glob(path+"*.html") if file not in exclude]
totaldomc = set()
for state in tqdm(states):
    tree,nodes = generateTree(state)
    tree = Tree.from_text(tree)
    taglist = traverseTree(tree)
    soup = makeTree(state)
    for tags in taglist:
        for tag in soup.find_all(tags):
            totaldomc.add(tag)
len(totaldomc)

100%|██████████| 38/38 [00:01<00:00, 29.67it/s]


2652

In [20]:
count = 0
with open("qpaths.json") as json_file:
    data = json.load(json_file)

for k in data.keys():
    count+=len(data[k])
count

642

In [21]:
count = 0
with open("cpaths.json") as json_file:
    data = json.load(json_file)

for k in data.keys():
    count+=len(data[k])
count

125

In [3]:
makeGraph_Qexplore("./Q.map")

(<networkx.classes.digraph.DiGraph at 0x7f1bde3d0910>,
 {'2a37943c3d7117b3f16609f866c1e35f': 'index1',
  '847671af6b9932afb5ca6a6be6665014': 'S2',
  '7f95da930eded8d67c2a66a139d54c52': 'S3',
  'ce711189b143057672386a81f2e24a33': 'S4',
  'fd4e47ef419b2d155085d61bc76feb71': 'S5',
  '39fc5ed66ef73c54991033109151e215': 'S6',
  '6775b8bf1e3ad860c196e0b671d740b0': 'S7',
  '9018ed66f9f0621005aac3def19c20c5': 'S8',
  '41aec5b7218b977368184c1830f6473e': 'S9',
  'fe6a1f9b49cd7e42af514a485b2005eb': 'S10',
  '253601522ecc33e9a99e07c292899dae': 'S11',
  '993c7c4884292b9f74568547a1397045': 'S12',
  '518c835e8f0e491a3e1e760d98c68e97': 'S13',
  '0e0e7781089d08f8a7699b6443370798': 'S14',
  '62577da006d53d02c81f1a8fbbb17c9d': 'S15',
  '1641ed534e2f6c6254c2147ad44f190d': 'S16'},
 {'index1': '2a37943c3d7117b3f16609f866c1e35f',
  'S2': '847671af6b9932afb5ca6a6be6665014',
  'S3': '7f95da930eded8d67c2a66a139d54c52',
  'S4': 'ce711189b143057672386a81f2e24a33',
  'S5': 'fd4e47ef419b2d155085d61bc76feb71',
  'S6

In [15]:
A = [{"src": "2a37943c3d7117b3f16609f866c1e35f", "edges": [{"action": "input!@!employee_passwd!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!remember_me!@!1!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://httpd.apache.org/", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://php.net/", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://timeclock.sourceforge.net/", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!employee_passwd!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!remember_me!@!1!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!employee_passwd!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!remember_me!@!1!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!employee_passwd!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!remember_me!@!1!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!remember_me!@!1!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!employee_passwd!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}], "url": "http://192.168.1.68/timeclock/timeclock.php", "start": 1}, {"src": "847671af6b9932afb5ca6a6be6665014", "edges": [{"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "7f95da930eded8d67c2a66a139d54c52"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_userid!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!login_password!@!nan!@!nan", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!nan!@!Log In!@!nan", "state": "7f95da930eded8d67c2a66a139d54c52"}], "url": "http://192.168.1.68/timeclock/login.php", "start": 0}, {"src": "7f95da930eded8d67c2a66a139d54c52", "edges": [{"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "7f95da930eded8d67c2a66a139d54c52"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "7f95da930eded8d67c2a66a139d54c52"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "fd4e47ef419b2d155085d61bc76feb71"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "fd4e47ef419b2d155085d61bc76feb71"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/logout.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}], "url": "http://192.168.1.68/timeclock/admin/index.php", "start": 0}, {"src": "ce711189b143057672386a81f2e24a33", "edges": [{"action": "input!@!employee_passwd!@!nan!@!nan", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "input!@!remember_me!@!1!@!nan", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "ce711189b143057672386a81f2e24a33"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "7f95da930eded8d67c2a66a139d54c52"}], "url": "http://192.168.1.68/timeclock/timeclock.php", "start": 0}, {"src": "fd4e47ef419b2d155085d61bc76feb71", "edges": [{"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "fd4e47ef419b2d155085d61bc76feb71"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "fd4e47ef419b2d155085d61bc76feb71"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "7f95da930eded8d67c2a66a139d54c52"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "fd4e47ef419b2d155085d61bc76feb71"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "fd4e47ef419b2d155085d61bc76feb71"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/logout.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}], "url": "http://192.168.1.68/timeclock/punchclock/menu.php", "start": 0}, {"src": "39fc5ed66ef73c54991033109151e215", "edges": [{"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://httpd.apache.org/", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://php.net/", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://timeclock.sourceforge.net/", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}], "url": "http://192.168.1.68/timeclock/punchclock/menu.php", "start": 0}, {"src": "6775b8bf1e3ad860c196e0b671d740b0", "edges": [{"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!csv!@!1!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!csv!@!0!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!submit!@!Edit Time!@!nan", "state": "62577da006d53d02c81f1a8fbbb17c9d"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php#", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "a!@!nan!@!nan!@!http://httpd.apache.org/", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "a!@!nan!@!nan!@!http://php.net/", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "a!@!nan!@!nan!@!http://timeclock.sourceforge.net/", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "select!@!office_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!group_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!user_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!csv!@!1!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!csv!@!0!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!office_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!group_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!user_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "select!@!office_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!group_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "select!@!user_name!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!csv!@!1!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "input!@!csv!@!0!@!nan", "state": "6775b8bf1e3ad860c196e0b671d740b0"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php#", "state": "253601522ecc33e9a99e07c292899dae"}], "url": "http://192.168.1.68/timeclock/reports/timerpt.php", "start": 0}, {"src": "9018ed66f9f0621005aac3def19c20c5", "edges": [{"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!csv!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!csv!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_paginate!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_paginate!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_show_details!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_show_details!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!2!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!3!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!4!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!5!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!submit!@!Edit Time!@!nan", "state": "1641ed534e2f6c6254c2147ad44f190d"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php#", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "a!@!nan!@!nan!@!http://httpd.apache.org/", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "a!@!nan!@!nan!@!http://php.net/", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "a!@!nan!@!nan!@!http://timeclock.sourceforge.net/", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "select!@!office_name!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "select!@!group_name!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "select!@!user_name!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_paginate!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_paginate!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_show_details!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_show_details!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!2!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!3!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!4!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!5!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_round_time!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!csv!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!csv!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "select!@!office_name!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "select!@!group_name!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "select!@!user_name!@!nan!@!nan", "state": "9018ed66f9f0621005aac3def19c20c5"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}], "url": "http://192.168.1.68/timeclock/reports/total_hours.php", "start": 0}, {"src": "41aec5b7218b977368184c1830f6473e", "edges": [{"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!1!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!0!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!submit!@!Edit Time!@!nan", "state": "0e0e7781089d08f8a7699b6443370798"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php#", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "a!@!nan!@!nan!@!http://httpd.apache.org/", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "a!@!nan!@!nan!@!http://php.net/", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "a!@!nan!@!nan!@!http://timeclock.sourceforge.net/", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!1!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!0!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!1!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!0!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!1!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "input!@!csv!@!0!@!nan", "state": "41aec5b7218b977368184c1830f6473e"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php#", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}], "url": "http://192.168.1.68/timeclock/reports/audit.php", "start": 0}, {"src": "fe6a1f9b49cd7e42af514a485b2005eb", "edges": [{"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "input!@!csv!@!1!@!nan", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "input!@!csv!@!0!@!nan", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "input!@!submit!@!Edit Time!@!nan", "state": "0e0e7781089d08f8a7699b6443370798"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "fe6a1f9b49cd7e42af514a485b2005eb"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}], "url": "http://192.168.1.68/timeclock/reports/audit.php", "start": 0}, {"src": "253601522ecc33e9a99e07c292899dae", "edges": [{"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!csv!@!1!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!csv!@!0!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "input!@!submit!@!Edit Time!@!nan", "state": "62577da006d53d02c81f1a8fbbb17c9d"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "253601522ecc33e9a99e07c292899dae"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}], "url": "http://192.168.1.68/timeclock/reports/timerpt.php", "start": 0}, {"src": "993c7c4884292b9f74568547a1397045", "edges": [{"action": "input!@!date_format!@!M/d/yyyy!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!from_date!@!nan!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!to_date!@!nan!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!csv!@!1!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!csv!@!0!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_paginate!@!1!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_paginate!@!0!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_show_details!@!1!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_show_details!@!0!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_display_ip!@!1!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_display_ip!@!0!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_round_time!@!1!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_round_time!@!2!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_round_time!@!3!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_round_time!@!4!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_round_time!@!5!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!tmp_round_time!@!0!@!nan", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "input!@!submit!@!Edit Time!@!nan", "state": "1641ed534e2f6c6254c2147ad44f190d"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "993c7c4884292b9f74568547a1397045"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}], "url": "http://192.168.1.68/timeclock/reports/total_hours.php", "start": 0}, {"src": "518c835e8f0e491a3e1e760d98c68e97", "edges": [{"action": "input!@!employee_passwd!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!reset_cookie!@!1!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php", "state": "2a37943c3d7117b3f16609f866c1e35f"}, {"action": "a!@!nan!@!nan!@!http://www.historychannel.com/tdih", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php", "state": "847671af6b9932afb5ca6a6be6665014"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php", "state": "39fc5ed66ef73c54991033109151e215"}, {"action": "a!@!nan!@!nan!@!http://httpd.apache.org/", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://mysql.org/", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://php.net/", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "a!@!nan!@!nan!@!http://timeclock.sourceforge.net/", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!reset_cookie!@!1!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!employee_passwd!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!reset_cookie!@!1!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "select!@!left_fullname!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "select!@!left_inout!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!employee_passwd!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!left_notes!@!nan!@!nan", "state": "518c835e8f0e491a3e1e760d98c68e97"}, {"action": "input!@!submit_button!@!Submit!@!nan", "state": "2a37943c3d7117b3f16609f866c1e35f"}], "url": "http://192.168.1.68/timeclock/timeclock.php", "start": 0}]

In [16]:
bb = set()
for a in A:
    for e in a['edges']:
        bb.add(e["action"])
bb

{'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/index.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/login.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/logout.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/punchclock/menu.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/audit.php#',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/index.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/timerpt.php#',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php',
 'a!@!nan!@!nan!@!http://192.168.1.68/timeclock/reports/total_hours.php#',
 'a!@!nan!@!nan!@!http://httpd.apache.org/',
 'a!@!nan!@!nan!@!http://mysql.org/',
 'a!@!nan!@!nan!@!http://php.net/',
 'a!@!nan!@!nan!@!http://timeclock.sourceforge.net/',
 'a!@!nan!@!nan!@!http://www.historychannel.com/tdih',
 'input!@!csv!@!0!@!nan',


In [17]:
len(bb)

47